In [1]:
import jupyter_black

jupyter_black.load()

In [2]:
# Sources:
# linux laptop
# amazon.de
# apple new
# apple refurbished
# top notebookcheck multimedia laptop
# x-kom
# delkom
# techlord
# allegro
# refurbished
# huawei laptop
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

pd.set_option("display.max_columns", None)
raw_laptops_data = pd.read_csv("laptops.csv")
raw_laptops_data.head()

,URL,Laptop Name,Brand,Brand Reliability,CPU,CPU P,GPU,GPU P,VRAM,RAM,Screen Size,Resolution,System,System S,Weight,Storage,Temp,New,Price
0,https://www.apple.com/pl/shop/buy-mac/macbook-...,MacBook Pro,Apple,0.70%,M4 Max,78.3,M3 Pro 14-Core GPU,66.65,NaN,48,16.2,7720704,MacOS,50,2.15,2000,42.8,100,23499
1,https://www.amazon.de/-/en/2023-Gram-Inch-Ultr...,LG Gram,LG,1.60%,i7-1360P,49.3,Iris Xe Graphics G7 96EUs,13.60,NaN,32,17.0,4096000,Windows,0,1.35,2000,43.9,100,6809
2,https://www.x-kom.pl/p/1246865-notebook-laptop...,ASUS Vivobook S15,Asus,3.20%,Qualcomm Snapdragon X Elite X1E-78-100,44.0,Snapdragon X Adreno X1-85,10.50,NaN,32,15.6,4665600,Windows,0,1.42,1000,43.6,100,4899
3,https://system76.com/laptops/darp10/configure,Darter Pro,System 76,NaN,Ultra 7 155H,50.9,Arc 8-Core iGPU,25.10,NaN,32,16.0,2304000,Linux,100,1.93,1000,NaN,100,6748
4,NaN,Legion 5 Pro-16,Lenovo,3.10%,Ryzen 5 5600H,37.0,RTX 3050 4GB Laptop GPU,32.60,NaN,16,16.0,4096000,Linux,100,2.45,512,39.9,100,4999


In [3]:
system_score = {
    "Windows": 0,
    "MacOS": 0.5,
    "Linux": 1,
    "NoOS": 1,
}

In [4]:
features = [
    {"name": "CPU Performance", "weight": 0.20, "invert": False},
    {"name": "GPU Performance", "weight": 0.20, "invert": False},
    {"name": "RAM", "weight": 0.15, "invert": False},
    {
        "name": "Effective VRAM",
        "weight": 0.15,
        "invert": False,
    },  # Updated to Effective VRAM
    {"name": "New", "weight": 0.09, "invert": False},
    {"name": "Storage", "weight": 0.05, "invert": False},
    {"name": "System", "weight": 0.04, "invert": False},
    {"name": "Weight", "weight": 0.04, "invert": True},  # Lower weight is better
    {"name": "Resolution", "weight": 0.04, "invert": False},
    {"name": "Temperature", "weight": 0.02, "invert": True},  # Lower temp is better
    {"name": "Brand Reliability", "weight": 0.02, "invert": False},
]
print("Weights sum:", sum(f["weight"] for f in features))

Weights sum: 1.0000000000000002


In [5]:
laptops_data = pd.DataFrame()
laptops_data["Laptop Name"] = raw_laptops_data["Laptop Name"]
laptops_data["Brand Reliability"] = (
    1 - raw_laptops_data["Brand Reliability"].str.replace("%", "").astype(float) / 100
)
laptops_data["CPU Performance"] = pd.to_numeric(
    raw_laptops_data["CPU P"], errors="coerce"
)
laptops_data["GPU Performance"] = pd.to_numeric(
    raw_laptops_data["GPU P"], errors="coerce"
)
laptops_data["RAM"] = pd.to_numeric(raw_laptops_data["RAM"], errors="coerce")
laptops_data["VRAM"] = pd.to_numeric(raw_laptops_data.get("VRAM", 0), errors="coerce")
laptops_data["Screen Size"] = pd.to_numeric(
    raw_laptops_data["Screen Size"], errors="coerce"
)
laptops_data["Resolution"] = pd.to_numeric(
    raw_laptops_data["Resolution"], errors="coerce"
)
laptops_data["System"] = raw_laptops_data["System"]
laptops_data["Weight"] = pd.to_numeric(raw_laptops_data["Weight"], errors="coerce")
laptops_data["Storage"] = pd.to_numeric(raw_laptops_data["Storage"], errors="coerce")
laptops_data["Temperature"] = pd.to_numeric(raw_laptops_data["Temp"], errors="coerce")
laptops_data["New"] = pd.to_numeric(raw_laptops_data["New"], errors="coerce") / 100
laptops_data["Price"] = pd.to_numeric(raw_laptops_data["Price"], errors="coerce")
laptops_data["GPU"] = raw_laptops_data["GPU"]

In [9]:
def get_gpu_type(gpu_name):
    gpu_name = str(gpu_name).lower()
    if "nvidia" in gpu_name or "amd radeon" in gpu_name or "rtx" in gpu_name:
        return "Dedicated"
    elif "apple m" in gpu_name:
        return "Unified"
    else:
        return "Integrated"


laptops_data["GPU Type"] = laptops_data["GPU"].apply(get_gpu_type)
mask_dedicated = laptops_data["GPU Type"] == "Dedicated"
laptops_data.loc[mask_dedicated, "Effective VRAM"] = (
    laptops_data.loc[mask_dedicated, "VRAM"]
    + 0.5 * laptops_data.loc[mask_dedicated, "RAM"]
)
mask_integrated = laptops_data["GPU Type"] == "Integrated"
laptops_data.loc[mask_integrated, "Effective VRAM"] = (
    laptops_data.loc[mask_integrated, "RAM"] * 0.5
)
mask_unified = laptops_data["GPU Type"] == "Unified"
laptops_data.loc[mask_unified, "Effective VRAM"] = (
    laptops_data.loc[mask_unified, "RAM"] * 0.8
)

In [10]:
laptops_data

,Laptop Name,Brand Reliability,CPU Performance,GPU Performance,RAM,VRAM,Screen Size,Resolution,System,Weight,Storage,Temperature,New,Price,GPU,GPU Type,Effective VRAM
0,MacBook Pro,0.993,78.3,66.65,48,NaN,16.2,7720704,MacOS,2.15,2000,42.8,1.0,23499,M3 Pro 14-Core GPU,Integrated,24.0
1,LG Gram,0.984,49.3,13.60,32,NaN,17.0,4096000,Windows,1.35,2000,43.9,1.0,6809,Iris Xe Graphics G7 96EUs,Integrated,16.0
2,ASUS Vivobook S15,0.968,44.0,10.50,32,NaN,15.6,4665600,Windows,1.42,1000,43.6,1.0,4899,Snapdragon X Adreno X1-85,Integrated,16.0
3,Darter Pro,NaN,50.9,25.10,32,NaN,16.0,2304000,Linux,1.93,1000,NaN,1.0,6748,Arc 8-Core iGPU,Integrated,16.0
4,Legion 5 Pro-16,0.969,37.0,32.60,16,NaN,16.0,4096000,Linux,2.45,512,39.9,1.0,4999,RTX 3050 4GB Laptop GPU,Dedicated,NaN
5,Huawei Matebook 16,0.977,38.9,15.00,16,NaN,16.0,4233600,Windows,1.99,512,40.2,1.0,4548,Radeon RX Vega 8 (Ryzen 4000/5000),Integrated,8.0
6,Schenker,NaN,49.3,13.60,32,NaN,17.3,2073600,NoOS,2.25,1000,39.1,1.0,5191,Iris Xe Graphics G7 96EUs,Integrated,16.0
7,Lenovo Legion Slim,0.969,45.7,60.70,32,NaN,16.0,4096000,Windows,2.40,1000,40.6,1.0,6995,GeForce RTX 4060,Dedicated,NaN
8,ASUS VivoBook 15,0.962,17.2,9.10,8,NaN,15.6,2073600,Linux,1.90,256,NaN,1.0,2200,MX250,Integrated,4.0
9,Xiaomi Redmibook 16 Pro,NaN,50.9,25.10,32,NaN,16.0,5898240,Windows,1.88,1000,40.3,1.0,4635,Arc 8-Core iGPU,Integrated,16.0


In [ ]:
scaler = MinMaxScaler()
columns_to_scale = [
    f["name"]
    for f in features
    if f["name"] not in ["System", "New", "Weight", "Temperature", "Brand Reliability"]
]
laptops_data[[f"{col} Score" for col in columns_to_scale]] = scaler.fit_transform(
    laptops_data[columns_to_scale]
)

In [ ]:
laptops_data["Weight Score"] = 1 - scaler.fit_transform(laptops_data[["Weight"]])
laptops_data["Temperature Score"] = 1 - scaler.fit_transform(
    laptops_data[["Temperature"]]
)
laptops_data["Brand Reliability Score"] = scaler.fit_transform(
    laptops_data[["Brand Reliability"]]
)
laptops_data["New Score"] = laptops_data["New"]
laptops_data["System Score"] = laptops_data["System"].map(system_score)

In [ ]:
laptops_data["Brand Reliability Score"] = laptops_data[
    "Brand Reliability Score"
].fillna(laptops_data["Brand Reliability Score"].mean())
laptops_data["Temperature Score"] = laptops_data["Temperature Score"].fillna(
    laptops_data["Temperature Score"].mean()
)

In [ ]:
laptops_data["Score"] = 0
for feature in features:
    score_col = f"{feature['name']} Score"
    laptops_data["Score"] += laptops_data[score_col] * feature["weight"]

laptops_data["Total Score"] = (laptops_data["Score"] / laptops_data["Price"]) * 10000
laptops_data = laptops_data.round(2)
laptops_data = laptops_data.sort_values(by=["Total Score", "Score"], ascending=False)

In [ ]:
laptops_data